# Импорт библиотек и авторизация в СПАРК

In [25]:
from contextlib import contextmanager
from typing import Optional
import xmlschema  # type: ignore
from zeep import Client  # type: ignore
import xml.etree.ElementTree as ET
import pandas as pd

#Подключение к контуру

wdsl_url = 'https://demoapi.spark-interfax.ru/'
client = Client(wdsl_url)

#Авторизация
login = 'PEKGate'
password = 'UesA9zB'

client.service.Authmethod(login, password)

'True'

# Чтение собранных селлеров

In [26]:
data = pd.read_excel(r"")
data

FileNotFoundError: [Errno 2] No such file or directory: ''

# Получение региона

In [27]:
def get_region(inn):
    inn = str(inn)
    if len(inn) == 10:#ЮР. ЛИЦА
        try:
            company = client.service.GetCompanyExtendedReport('', inn, '')
            company = str(company)
            company = company[64:-3]
            try:
                root = ET.fromstring(company)
                reg = root.find('.//OKATO').get('RegionName')
            except:
                reg = 'регион не найден'
        except:
            reg = 'компания не найдена в СПАРК'
    elif len(inn) == 12:#ФИЗ. ЛИЗА
        try:
            enterpreneur = client.service.GetEntrepreneurReport('', '', inn)
            enterpreneur = str(enterpreneur)
            if enterpreneur[38:42] != 'True':
                raise Exception
            enterpreneur = enterpreneur[61:-3]
            try:
                root = ET.fromstring(enterpreneur)
                reg = root.find('.//OKATO').get('RegionName')
            except:
                reg = 'Регион не найден'
        except:
            reg = 'Предприниматель не найден в СПАРК'
    else:#НЕПРАВИЛЬНЫЕ
        #print('Неправильный инн')
        reg = 'Непр'

    return reg

# Получение телефонов

In [28]:
def get_phones(inn):
    inn = str(inn)
    if len(inn) == 10:#Юр. лица
        try:
            company = client.service.GetCompanyExtendedReport('', inn, '')
            company = str(company)
            company = company[64:-3]
            try:
                root = ET.fromstring(company)
                phones = root.findall('.//Phone')
                phone_number_list = []
                for phone in phones:
                    phone_number = phone.get('Code') + phone.get('Number')
                    phone_number_list.append(phone_number)

                phone_number_list = ", ".join(phone_number_list)
            except:
                phone_number_list = 'телефоны не найдены'
        except:
            phone_number_list = 'компания не найдена в СПАРК'
    elif len(inn) == 12:#Физ. лиза
        
        try:
            enterpreneur = client.service.GetEntrepreneurReport('', '', inn)
            enterpreneur = str(enterpreneur)
            if enterpreneur[38:42] != 'True':
                raise Exception
            enterpreneur = enterpreneur[61:-3]
            try:
                root = ET.fromstring(enterpreneur)
                phones = root.findall('.//Phone')
                phone_number_list = []
                for phone in phones:
                    phone_number = phone.get('Code') + phone.get('Number')
                    phone_number_list.append(phone_number)

                phone_number_list = ", ".join(phone_number_list)
            except:
                phone_number_list = 'Телефоны не найдены'
        except:
            phone_number_list = 'Предприниматель не найден в СПАРК'
        
        #phone_number_list = 'Физ. лицо'
    else:#Неправильные
        #print('Неправильный инн')
        phone_number_list = 'Непр'

    return phone_number_list

# Применение к датасету

In [29]:
data['Регион'] = data['ИНН'].apply(get_region)
data['Телефоны'] = data['ИНН'].apply(get_phones)
data

,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Регион,Телефоны
0,616507365740,https://www.wildberries.ru/seller/1266726,Top-shop,снэки,6 066,Предприниматель не найден в СПАРК,Предприниматель не найден в СПАРК
1,370226677848,https://www.wildberries.ru/seller/1177407,FORCE-store,снэки,10 957,Ивановская область,
2,7729422282,https://www.wildberries.ru/seller/ast,Издательство АСТ,снэки,30 116 512,Москва,
3,540531258600,https://www.wildberries.ru/seller/1157796,VALENTINA,снэки,20 485,Предприниматель не найден в СПАРК,Предприниматель не найден в СПАРК
4,711709404177,https://www.wildberries.ru/seller/82331,PROFI STAR,снэки,155 949,Тульская область,
...,...,...,...,...,...,...,...
191,180301181770,https://www.wildberries.ru/seller/597325,Camellia house,снэки,11 348,Удмуртская Республика,
192,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,снэки,74 092,Предприниматель не найден в СПАРК,Предприниматель не найден в СПАРК
193,7707030771,https://www.wildberries.ru/seller/32693,Официальный магазин ErichKrause®,снэки,14 051 515,Москва,"4992343794, 9037554204, 9037730932, 9039714482"
194,121526972341,https://www.wildberries.ru/seller/1202545,DANY FASHION,снэки,17 840,Предприниматель не найден в СПАРК,Предприниматель не найден в СПАРК


# Распределение по регионам

In [30]:
regs = data.pivot_table('ИНН', 'Регион', aggfunc='count')
regs.sort_values(by='ИНН', ascending=False)

,ИНН
Регион,
Предприниматель не найден в СПАРК,56
Москва,40
Непр,28
Московская область,12
Свердловская область,6
Ивановская область,6
Новосибирская область,5
Санкт-Петербург,5
Владимирская область,3


# Сохранение

In [32]:
data['ИНН'] = data['ИНН'].apply(str)
data.to_excel(r"")

In [127]:
regs.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Первая партия\Распределение по регионам.xlsx")

# Закрытие Сессии

In [31]:
client.service.End()

True